# Extra: XGBoost

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import Counter
from IPython.core.interactiveshell import InteractiveShell
from sklearn.decomposition import PCA
from pandas import read_csv
from sklearn.neighbors import KNeighborsClassifier, LocalOutlierFactor
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import (
    train_test_split, cross_val_score, StratifiedKFold, GridSearchCV, KFold,
    cross_validate
)
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from statsmodels.genmod.generalized_linear_model import GLM
from pandas.plotting import scatter_matrix
import scipy.stats as stats
from scipy.stats import boxcox
from pandas.api.types import CategoricalDtype

from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
)
from sklearn.naive_bayes import BernoulliNB, GaussianNB, CategoricalNB
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, f1_score
)
from IPython.core.interactiveshell import InteractiveShell
from sklearn.tree import DecisionTreeClassifier
#import graphviz
from sklearn.tree import export_graphviz
from sklearn.ensemble import (
    RandomForestClassifier, VotingClassifier, GradientBoostingClassifier,
    StackingClassifier, ExtraTreesClassifier
)
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from time import time
from datetime import timedelta
from xgboost import XGBClassifier


InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.precision', 3)
sns.set()

import warnings
warnings.filterwarnings('ignore')

In [18]:
DF3 = pd.read_pickle('RiskFactors_processed.pkl.bz2',compression='bz2')
selected_features = ['Age', 'Number of sexual partners', 'First sexual intercourse', 'Smokes', 
                     'Hormonal Contraceptives', 'IUD', 'STDs:condylomatosis', 'STDs:HIV', 'Dx:Cancer', 'Dx:CIN', 'Dx:HPV', 'Cancer'] # Lasso extraction
DF3 = DF3[selected_features]
DF3 = DF3.drop(columns=['STDs:HIV', 'Dx:CIN','Dx:Cancer','Dx:HPV']) # We extract features from ExtraTrees
DF3.head()
DF3.shape
numerical_columns = ['Age', 'Number of sexual partners', 'First sexual intercourse']
categorical_columns = ['Smokes', 'Hormonal Contraceptives', 'STDs:condylomatosis', 'IUD']

,Age,Number of sexual partners,First sexual intercourse,Smokes,Hormonal Contraceptives,IUD,STDs:condylomatosis,Cancer
0,0.901,0.758,0.539,0,1,0,0,0
1,0.746,0.851,0.539,1,1,0,0,0
2,0.514,0.314,0.804,0,1,0,0,0
3,0.685,0.506,0.804,1,1,0,0,1
4,0.663,0.314,0.652,0,1,0,0,0


(662, 8)

In [19]:
def compute_metrics(y_real,y_pred):
    # By default it will compute the binary recall of class 1, we can specify which class do we want by using this parameter 
    recall_class_1 =recall_score(y_real,y_pred, pos_label=1)
    f1_class_1 =f1_score(y_real,y_pred, pos_label=1)
    accuracy = accuracy_score(y_real,y_pred)
    f1_macro =f1_score(y_real,y_pred, average='macro')
    precison_macro =precision_score(y_real,y_pred,  average='macro')
    recall_macro =recall_score(y_real,y_pred,  average='macro')
    return [recall_class_1, f1_class_1, accuracy,f1_macro,precison_macro,recall_macro ]

In [20]:
def confusion(true, pred):
    """
    Function for pretty printing confusion matrices
    """
    true.name = 'target'
    pred.name = 'predicted'
    cm = pd.crosstab(true.reset_index(drop=True), pred.reset_index(drop=True))
    cm = cm[cm.index]
    return cm

In [21]:
# Load the dataset and split into train, test, and validation sets
X = DF3.iloc[:, :-1]  # Features (all columns except the last one)
y = DF3['Cancer']  # Target variable (last column)

np.random.seed(42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)

# Apply resampling techniques to handle class imbalance
undersampler = RandomUnderSampler(sampling_strategy=0.7,random_state=40)  # Adjust the sampling strategy as needed
X_train_undersampled, y_train_undersampled = undersampler.fit_resample(X_train, y_train)
X_train.shape, X_val.shape, X_test.shape

((396, 7), (133, 7), (133, 7))

In [22]:
# Train the XGBoost model
xgb = XGBClassifier(scale_pos_weight=(len(X_train) / len(X_train_undersampled))) # adresses imbalance
xgb.fit(X_train_undersampled, y_train_undersampled)

# Predict on the validation set
y_pred_val = xgb.predict(X_val)

results = pd.DataFrame(index=[], columns= ['**Recall class 1**', '**F1 class 1**','Accuracy', 'F1 Macro', 'Precision Macro', 'Recall Macro'])
results.loc['xgboost', :] = compute_metrics(y_val, y_pred_val);
results.sort_values(by='**Recall class 1**', ascending=False)
confusion(pd.Series(y_val), pd.Series(y_pred_val))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

,**Recall class 1**,**F1 class 1**,Accuracy,F1 Macro,Precision Macro,Recall Macro
xgboost,0.353,0.169,0.556,0.433,0.486,0.47


predicted,0,1
target,,
0,68,48
1,11,6
